In [ ]:
import aiohttp
import asyncio


endpoint = 'https://api.idg.vnpt.vn/tts-service/v1/standard'
endpoint_status = 'https://api.idg.vnpt.vn/tts-service/v1/check-status'
authorization = ''
token_id = ''
token_key = ''
headers_auth = {
    'authorization': authorization,
    'token-id': token_id,
    'token-key': token_key,
}
interval_result_check = 1 # in seconds, should set to 10 or 30 to avoid overloading the server

audio_format = 'mp3' # or 'wav' (lossless but more heavy)



async def call_api_2_step(session: aiohttp.ClientSession, text: str):
    try:
        payload = {
            "text": text,
            "model": "news",
            "region": "female_north",
            "audio_format": audio_format,
        }
        # step 1
        async with session.post(endpoint, headers=headers_auth, json=payload) as response:
            result = await response.json()
            text_id = result.get('object',{}).get('text_id')
        if not text_id: # or not success
            return result

        # step 2: check status for result every interval
        result = await check_result(session, text_id)
        while result.get('object',{}).get('code') == 'pending':
            await asyncio.sleep(interval_result_check)
            result = await check_result(session, text_id)

    except Exception as e:
        # custom your error handling here
        result = {'message': str(e)}
    return result

async def check_result(session: aiohttp.ClientSession, text_id: str):
    payload = {'text_id': text_id}
    async with session.post(endpoint_status, headers=headers_auth, json=payload) as response:
        result = await response.json()
    return result

async def run_async_many(texts: list[str]):
    async with aiohttp.ClientSession() as session:
        tasks = [call_api_2_step(session, text) for text in texts]
        results = await asyncio.gather(*tasks, return_exceptions=True)
    return results

async def download_audio(url: str, save_path: str):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            audio = await response.read()
    with open(save_path,'wb') as f:
        f.write(audio)

In [2]:
import IPython.display

def display_audio(url):
    audio = IPython.display.Audio(url=url)
    IPython.display.display(audio)

In [3]:
text = 'alo 1, 2, 3'
results = await run_async_many([text])
results

[{'message': 'IDG-00000000',
  'object': {'ai_version': '2.3',
   'code': 'success',
   'playlist': [{'audio_link': 'https://voice-storage.vnpt.vn/text-speech/20250224/tts/42/750cd412f0d149aaa3d34f470ee8f304.mp3',
     'idx': '1',
     'text': 'alo 1, 2, 3',
     'text_len': 11,
     'total': '1',
     'word_timepoint': []}],
   'r_audio_full': '',
   'r_audio_full_finished': False,
   'text_id': '78c7ede442e92a232ba8805c8910cc7b',
   'text_len': 11,
   'version': '1.0.0'}}]

In [4]:
link = results[0]['object']['playlist'][0]['audio_link']
display_audio(link)

In [5]:
await download_audio(link, f'tmp.{audio_format}')